In [ ]:
# fabric ssh setup
from fabric import Connection


classic_sender = Connection(
    host='pc538.emulab.net',
    user = 'PeterYao',
    port=22,
)




prague_sender = Connection(
    host='pc549.emulab.net',
    user = 'PeterYao',
    port=22,
)


dualq = Connection(
    host='pc557.emulab.net',
    user='PeterYao',
    port=22,
)

fifo_deep = Connection(
    host='pc524.emulab.net',
    user
    = 'PeterYao',  
    port=22,
)

rx = Connection(
    host='pc760.emulab.net',
    user = 'PeterYao',
    port=22,
)

classic_receiver = Connection(
    host='pc536.emulab.net',
    user = 'PeterYao',
    port = 25674,
)

prague_receiver = Connection(
    host='pc536.emulab.net',
    user =  'PeterYao',
    port= 25675,
)

connections = [classic_sender, classic_receiver, prague_sender, dualq, fifo_deep, rx, prague_receiver]

In [ ]:
# l4s Installation
for node in connections[2:]:
    node.run("wget https://github.com/L4STeam/linux/releases/download/testing-build/l4s-testing.zip")
    node.sudo("apt install unzip")
    node.run("unzip l4s-testing.zip")
    
    node.sudo("dpkg --install debian_build/*")
    node.sudo("update-grub")
    node.sudo("reboot", warn=True)    

In [ ]:
# Verify L4S
for node in connections:
    node.run("hostname; uname -a")

In [ ]:
# Modprobe TCP prague
for node in connections[2:]:
    # node.sudo("modprobe sch_dualpi2")
    node.sudo("modprobe tcp_prague")

In [ ]:
# Setup tcp prague
classic_sender.sudo("sysctl -w net.ipv4.tcp_congestion_control=cubic")
classic_sender.sudo("sysctl -w net.ipv4.tcp_ecn=1")
prague_sender.sudo("sysctl -w net.ipv4.tcp_congestion_control=prague")
prague_sender.sudo("sysctl -w net.ipv4.tcp_ecn=3")

In [ ]:
# Install DualPI2
cmd_dualpi2="""sudo apt-get update
sudo apt -y install git gcc make bison flex libdb-dev libelf-dev pkg-config libbpf-dev libmnl-dev libcap-dev libatm1-dev selinux-utils libselinux1-dev
sudo git clone https://github.com/L4STeam/iproute2.git
cd iproute2
sudo chmod +x configure
sudo ./configure
sudo make
sudo make install"""
dualq.sudo(cmd_dualpi2)
dualq.sudo("modprobe sch_dualpi2")